In [4]:
import sys
    
import numpy as np

import os

import plotly.graph_objects as go

try:
    import nPDyn
    from nPDyn.models.params import Parameters
    from nPDyn.models.model import Model, Component
except ModuleNotFoundError or ImportError:
    !{sys.executable} -m pip install git+https://github.com/kpounot/nPDyn@dev_lmfit#egg=nPDyn

    import nPDyn
    from nPDyn.models.params import Parameters
    from nPDyn.models.model import Model, Component
    

'v2.0.1'

## Instantiating a model, here a weighed pseudo-Voigt profile with linear background

In [2]:
X = np.arange(-30, 30, 0.01)
q = np.arange(0.2, 2, 0.15)[:, np.newaxis]

# define some lineshapes
def lorentzian(x, amplitude, sigma, center):
    return amplitude / np.pi * sigma / ((x - center)**2 + sigma**2)

def gaussian(x, amplitude, sigma, center):
    return amplitude / np.sqrt(2*np.pi*sigma**2) * np.exp(-(x - center)**2 / (2 * sigma**2))

def hline(x, a, b):
    return a * x + b

# initialize a Parameters instance
# the 'amplitude' is set using a tuple of type '(value, bounds)'
# the 'sigma' and 'center' are set using a dictionary
p = Parameters(
    amplitude=(np.zeros_like(q) + 0.2, (0., np.inf)),
    sigma={'value': 0.5, 'bounds': (0., np.inf)},
    center={'value': 0.5, 'fixed': True})

# create a first model that contains the pseudo-voigt profile
# the first component is added using the 'addComponent' method, the second usint the 'add' operator
m = Model(p)
m.addComponent(Component('Lorentzian',
                         lorentzian, 
                         amplitude='np.exp(-1.2*q**2/6) * amplitude', 
                         sigma='sigma'))

m = m + Component('Gaussian',
                  gaussian, 
                  amplitude='1 - np.exp(-1.2*q**2/6) * amplitude',
                  sigma='sigma / np.sqrt(2 * np.log(2))')

# the background could be added directly in 'm', but a second model is created here for demonstration purpose
bkgd = Component('Background', hline, a=0.5e-4, b=0.006)
m2 = Model(p)
m2 += m + bkgd

## Plotting the model

In [3]:
fig = go.FigureWidget(layout=go.Layout(xaxis={'title': 'energy E'},
                                       yaxis={'title': 'S(q, E)', 'type':'log'}))

for idx, val in enumerate(m2.eval(X, q=q)):
    fig.add_trace(go.Scatter(x=X, y=val, name="q = %.2f" % q[idx]))

fig.show()


No fitted parameters found for the sub-model: Model at 0x7fdb26faab38.
Using initial parameters instead.


NameError: name 'q' is not defined